In [3]:
import pandas as pd
import matplotlib as mp
import numpy as np

In [4]:
df1 = pd.read_excel('Coffee_Sales_Coffee_De_Madrid.xlsx')
df2 = pd.read_excel('Coffee_Sales_CoffeeLab.xlsx')
df3 = pd.read_excel('Coffee_Sales_Arabica.xlsx')

In [5]:
# Объединяю все данные в одну таблицу


df = pd.concat([df1, df2, df3], ignore_index=True)

In [6]:
df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,store_name
0,106,2023-01-01,11:01:48,1.0,3,Cay yolu,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm,Coffee De Madrid
1,107,2023-01-01,11:01:58,1.0,3,Cay-yolu,38,3.75,Coffee,Barista Espresso,Latte,Coffee De Madrid
2,108,2023-01-01,11:01:58,1.0,3,Cay yolu,75,3.50,Bakery,Pastry,Croissant,Coffee De Madrid
3,112,2023-01-01,11:08:11,1.0,3,Cay yolu,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,Coffee De Madrid
4,114,2023-01-01,11:09:01,1.0,3,Cay yolu,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,Coffee De Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...
156567,110166,2023-05-28,07:55:47,2.0,5,Kızlay,29,2.50,Coffee,Gourmet brewed coffee,Columbian Medium Roast Rg,Arabica
156568,48412,2023-03-22,07:15:42,1.0,5,Kizilay,26,3.00,Coffee,Organic brewed coffee,Brazilian Rg,Arabica
156569,89277,2023-05-09,09:25:54,1.0,5,Kizilay,37,3.00,Coffee,Barista Espresso,Espresso shot,Arabica
156570,29883,2023-02-22,10:28:33,2.0,5,NaN,72,2.65,Bakery,Scone,Ginger Scone,Arabica


In [8]:
# Удаляю дубликаты


df = df.drop_duplicates()

In [9]:
# Смотрю на уникальные значения в столбцах где есть проблемы


df.store_location.unique()

array(['Cay yolu', 'Cay-yolu', nan, 'Cayyolu', 'Cayolu', 'Bahcelievler',
       'Bahcievler', 'Bahcelivr', 'Bahceleiver', 'Kizilay', 'Kızlay',
       'Kzilay', 'Kizly'], dtype=object)

In [10]:
df.product_category.unique()

array(['Coffee', 'Bakery', 'Drinking Chocolate', 'Tea', nan, 'Loose Tea',
       'Coffee beans', 'Branded', 'Packaged Chocolate', 'Flavours'],
      dtype=object)

In [11]:
# Исправляю ошибки в названии локации


correct_store_locations = {
    'Kzilay': 'Kizilay', 'Kızlay': 'Kizilay', 'Kizly': 'Kizilay',
    'Bahcievler': 'Bahcelievler', 'Bahcelivr': 'Bahcelievler', 'Bahceleiver': 'Bahcelievler',
    'Cayolu': 'Cay yolu', 'Cayyolu': 'Cay yolu', 'Cay-yolu': 'Cay yolu'}


# Заменяем ошибочные значения на правильные


df['store_location'] = df['store_location'].replace(correct_store_locations)

/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/2694321088.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['store_location'] = df['store_location'].replace(correct_store_locations)


In [12]:
# Смотрю на количество пропусков в данных


df.isna().sum()

transaction_id         0
transaction_date       0
transaction_time       0
transaction_qty     7456
store_id               0
store_location      7456
product_id             0
unit_price          7456
product_category    7456
product_type           0
product_detail         0
store_name             0
dtype: int64

In [13]:
# Начинаю с столбца цены товара (unit_price)
# unit_price - это цена товара. Я знаю, что у некоторых товаров, которые включают в себя пропуск в цене,
# эта цена указана в других заказах. Следовательно выбранная стратегия для заполнения пропусков в цене - 
# вставить цену из заказа у которого одинаковый id товара (product_id)


df['unit_price'] = df.groupby('product_id')['unit_price'].transform(lambda x: x.fillna(x.iloc[0]))

/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/1693656786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unit_price'] = df.groupby('product_id')['unit_price'].transform(lambda x: x.fillna(x.iloc[0]))


In [17]:
# выведу значения количества проданных товаров за один заказ


df.transaction_qty.value_counts()

transaction_qty
1.0    82837
2.0    55682
3.0     3106
4.0       23
8.0       10
6.0        2
Name: count, dtype: int64

In [18]:
# здесь применю стратегию замены на самое популярное значение


df['transaction_qty'].fillna(1.0, inplace=True)

/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/655957337.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transaction_qty'].fillna(1.0, inplace=True)
/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/655957337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_qty'].fillna(1.0, inplace=T

In [14]:
# Теперь смотрю на пропуски в категориальных данных
# Выведу значения в столбце категории продукта (product_category)


df.product_category.value_counts()

product_category
Coffee                55527
Tea                   43139
Bakery                21640
Drinking Chocolate    10887
Flavours               6471
Coffee beans           1673
Loose Tea              1150
Branded                 711
Packaged Chocolate      462
Name: count, dtype: int64

In [15]:
# Использую ту же стратегию, что и с ценой товара, и вставлю ту же категорию что и в других заказах того же товара


df['product_category'] = df.groupby('product_id')['product_category'].transform(lambda x: x.fillna(x.iloc[0]))

/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/3800438454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_category'] = df.groupby('product_id')['product_category'].transform(lambda x: x.fillna(x.iloc[0]))


In [20]:
# Также заменю пропуски в названии локации (store_location) на самую популярную локацию для кофеин в городе


df['store_location'].fillna(df['store_location'].mode()[0], inplace=True)

/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/3730601764.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['store_location'].fillna(df['store_location'].mode()[0], inplace=True)
/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_51096/3730601764.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['store_locat

In [21]:
# Еще раз выведу пропуски в данных


df.isna().sum()

transaction_id        0
transaction_date      0
transaction_time      0
transaction_qty       0
store_id              0
store_location        0
product_id            0
unit_price          501
product_category    385
product_type          0
product_detail        0
store_name            0
dtype: int64

In [49]:
# Вижу что в таблицах цены и категории продукта (unit_price, product_category) все еще остались некоторые пропуски
# Пропуски в цене заменю средним значением цены всех товаров, а пропуски в категории самой популярной категорией


df['product_category'].fillna(df['product_category'].mode()[0], inplace=True)
df['unit_price'].fillna(df['unit_price'].median(), inplace=True)

/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_3525/384555420.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transaction_qty'].fillna(1.0, inplace=True)
/var/folders/f3/w_srs2_148q2vhs8t74vnypc0000gn/T/ipykernel_3525/384555420.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_qty'].fillna(1.0, inplace=Tru

In [22]:
# Выведу получившуюся таблицу


df

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,store_name
0,106,2023-01-01,11:01:48,1.0,3,Cay yolu,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm,Coffee De Madrid
1,107,2023-01-01,11:01:58,1.0,3,Cay yolu,38,3.75,Coffee,Barista Espresso,Latte,Coffee De Madrid
2,108,2023-01-01,11:01:58,1.0,3,Cay yolu,75,3.50,Bakery,Pastry,Croissant,Coffee De Madrid
3,112,2023-01-01,11:08:11,1.0,3,Cay yolu,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,Coffee De Madrid
4,114,2023-01-01,11:09:01,1.0,3,Cay yolu,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,Coffee De Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...
154120,149386,2023-06-30,18:28:56,2.0,5,Kizilay,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm,Arabica
154121,149387,2023-06-30,18:30:02,1.0,5,Kizilay,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,Arabica
154122,149388,2023-06-30,18:30:02,1.0,5,Kizilay,72,3.25,Bakery,Scone,Ginger Scone,Arabica
154123,149435,2023-06-30,19:40:09,2.0,5,Kizilay,58,3.50,Drinking Chocolate,Hot chocolate,Dark chocolate Rg,Arabica


In [23]:
# Сохраню датасет для дальнейшей работы


file_name = "Coffee_Sales_Ankara.xlsx"
df.to_excel(file_name, index=False)